In [1]:
#Import libraries
import tweepy
import pandas as pd
import time

In [3]:
%run ../keys.ipynb 

In [4]:
#https://www.youtube.com/watch?v=bNDRiaFyLrs
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,timeout=600000)

In [5]:
#https://docs.tweepy.org/en/stable/code_snippet.html#handling-the-rate-limit-using-cursors
#https://stackoverflow.com/questions/51700960/runtimeerror-generator-raised-stopiteration-every-time-i-try-to-run-app
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except (tweepy.RateLimitError, tweepy.TweepError):
            print("sleeping")
            time.sleep(15 * 60)    
        except StopIteration:
            return      

In [6]:
##https://docs.tweepy.org/en/latest/api.html#search-tweets
#Fetch the response for a given number of tweets where location is not null
def getTweets_by_hashTag(hashtag,num_tweets): 
    max_id = None
    tweets = []
    user_location = []
    created_at = []
    tweet_id = []
    while(len(tweets)<num_tweets): #loop iterates until tweets length < or = to num_tweets
        for item in limit_handled(tweepy.Cursor(api.search,
                              q = hashtag,
                              tweet_mode="extended",
                              include_entities=False,
                              lang="en",
                              result_type="recent",             
                             max_id=max_id).items(1000)):
            if item._json['user']['location'].strip(" ") != "" : #get post where "location" is not empty
                    tweets.append(item.full_text)
                    user_location.append(item._json['user']['location'])
                    created_at.append(item._json['created_at'])
                    tweet_id.append(item._json['id'])
                    max_id = tweet_id[-1]-1 #replace tweets with last recorded time stamp
        print(f"Returning {len(tweets)} tweets")
    tweet_df = pd.DataFrame({'tweets':tweets,'user_location':user_location,'tweet_posted_on':created_at,'tweet_id':tweet_id})        
    return tweet_df

### #workfromhome Data Collection

In [7]:
#get tweets of '#workfromhome' and save it in a dataframe
wfh_hashtag_df = getTweets_by_hashTag("#workfromhome",5000)

Returning 666 tweets
Returning 1356 tweets
sleeping
Returning 2102 tweets
Returning 2629 tweets
Returning 3330 tweets
sleeping
Returning 4007 tweets
Returning 4728 tweets
sleeping
Returning 5397 tweets


In [8]:
#Display the first five rows 
wfh_hashtag_df.head()

tweets  \
0  Quickly learn how to construct a really good c...   
1  Could YOU Fire Your Boss and Live the Dream? T...   
2  Are you exhausting yourself trying to make a s...   
3  RT @T1Vcollaborate: ThinkHub’s advanced #conte...   
4  RT @Linda_U_W_A: Uniquely Your Designs: Starti...   

                   user_location                 tweet_posted_on  \
0                London, England  Fri Jul 16 22:34:40 +0000 2021   
1       Chapala, Jalisco, Mexico  Fri Jul 16 22:28:19 +0000 2021   
2  Jalisco, México-retired Iowan  Fri Jul 16 22:21:11 +0000 2021   
3                    Raleigh, NC  Fri Jul 16 22:19:56 +0000 2021   
4             Kingston upon Hull  Fri Jul 16 22:19:38 +0000 2021   

              tweet_id  
0  1416164405633499136  
1  1416162808379678722  
2  1416161014496874498  
3  1416160698279895043  
4  1416160623587643399

In [9]:
#Save in a 'dataset' folder with a name 'tweet_df.csv'
wfh_hashtag_df.to_csv('../dataset/tweet_df.csv')

### #remotework Data Collection

In [10]:
#get tweets of '#remotework' and save it in a dataframe
remote_work = getTweets_by_hashTag("#remotework",5000)

Returning 640 tweets
Returning 1329 tweets
sleeping
Returning 2057 tweets
Returning 2680 tweets
sleeping
Returning 3315 tweets
Returning 4008 tweets
Returning 4754 tweets
sleeping
Returning 5429 tweets


In [12]:
#Display the first five rows 
remote_work.head()

tweets  \
0  🏡 #RemoteWork is on the rise &amp; slowly beco...   
1  Opportunity to join a fantastic team at a hi-t...   
2  Good news for #JobSeekers open to #RemoteWork!...   
3  Four Ways to Energize a Post-Pandemic Workforc...   
4  🚑 These are the tools that will save your #Rem...   

                    user_location                 tweet_posted_on  \
0                    New York, NY  Fri Jul 16 23:43:03 +0000 2021   
1               London | New York  Fri Jul 16 23:42:07 +0000 2021   
2  Sydney | Hong Kong | Singapore  Fri Jul 16 23:41:30 +0000 2021   
3                     Chicago, IL  Fri Jul 16 23:39:09 +0000 2021   
4                      Duluth, GA  Fri Jul 16 23:37:33 +0000 2021   

              tweet_id  
0  1416181616846811137  
1  1416181380279635970  
2  1416181225979473920  
3  1416180635903868934  
4  1416180231350759425

In [13]:
#Save in a 'dataset' folder with a name 'remote_work.csv'
remote_work.to_csv('../dataset/remote_work.csv')